In [4]:
# coding=utf-8
"""
This module contains the tests which ensure the the flashloan_tokens parameter is respected when using the b3_two_hop and bancor_v3 arb modes.
"""
import subprocess

import pytest

from fastlane_bot import Bot
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC

print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
SushiswapV2 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
Version = 3-b2.2 [requirements >= 3.0 is met]


# Setup

In [5]:
def find_main_py():
    # Start at the directory of the current script
    cwd = os.path.abspath(os.path.join(os.getcwd()))
    
    with open("log.txt", "w") as f:
        f.write(f"Searching for main.py in {cwd}")
                
    print(f"Searching for main.py in {cwd}")
    while True:
        # Check if main.py exists in the current directory
        if "main.py" in os.listdir(cwd):
            return cwd  # Found the directory containing main.py
        else:
            # If not, go up one directory
            new_cwd = os.path.dirname(cwd)

            # If we're already at the root directory, stop searching
            if new_cwd == cwd:
                raise FileNotFoundError("Could not find main.py in any parent directory")

            cwd = new_cwd
       
       
def run_command(mode):
    
    # Find the correct path to main.py
    main_script_path = find_main_py()
    print(f"Found main.py in {main_script_path}")
    main_script_path = main_script_path + "/main.py"

    # Run the command
    cmd = [
        "python",
        main_script_path,
        f"--arb_mode={mode}",
        "--default_min_profit_gas_token=60",
        "--limit_bancor3_flashloan_tokens=True",
        "--use_cached_events=True",
        "--logging_path=fastlane_bot/data/",
        "--timeout=60",
        "--alchemy_max_block_fetch=10",
        "--polling_interval=0",
        "--blockchain=ethereum"
    ]
    subprocess.Popen(cmd)
        
    # Wait for the expected log line to appear
    expected_log_line = "limiting flashloan_tokens to ["
    found = False
    result = subprocess.run(cmd, text=True, capture_output=True, check=True, timeout=140)

    # Check if the expected log line is in the output
    if expected_log_line in result.stderr:
        found = True

    if not found:
        pytest.fail("Expected log line was not found within 1 minute")  # If we reach this point, the test has failed

## Test Flashloan Tokens b3_two_hop

In [6]:
run_command("b3_two_hop")

Searching for main.py in /Users/mikewcasale/Documents/GitHub/bancorprotocol/fastlane-bot/resources/NBTest
Found main.py in /Users/mikewcasale/Documents/GitHub/bancorprotocol/fastlane-bot


2023-11-16 03:28:25,216 [fastlane:INFO] - 
2023-11-16 03:28:25,216 [fastlane:INFO] - **********************************************
2023-11-16 03:28:25,216 [fastlane:INFO] - The logging path is set to: fastlane_bot/data//logs/20231116-032825/bot.log
2023-11-16 03:28:25,216 [fastlane:INFO] - **********************************************
2023-11-16 03:28:25,216 [fastlane:INFO] - 
2023-11-16 03:28:25,834 [fastlane:INFO] - Using mainnet config
2023-11-16 03:28:25,898 [fastlane:INFO] - tokens: 35597, $CHAD-c918
2023-11-16 03:28:25,900 [fastlane:INFO] - unique_tokens: 35596
2023-11-16 03:28:25,913 [fastlane:INFO] - Flashloan tokens are set as: ['BNT-FF1C', 'WETH-6Cc2', 'USDC-eB48', 'USDT-1ec7', 'LINK-86CA']
2023-11-16 03:28:25,913 [fastlane:INFO] - Running data fetching for exchanges: ['carbon_v1', 'bancor_v3', 'bancor_v2', 'bancor_pol', 'uniswap_v3', 'uniswap_v2', 'sushiswap_v2', 'balancer', 'pancakeswap_v2', 'pancakeswap_v3']
2023-11-16 03:28:25,913 [fastlane:INFO] - 
        ++++++++++++